In [12]:
public interface ICommand {
    void Execute();
}

public class Button {
    public string Label { get; set; }
    private ICommand clickCommand;
    
    public Button (ICommand click) {
        this.clickCommand = click;
    }    

    public void Click () {
        clickCommand.Execute();
    }
}

public class CustomerService {
    public void AddCustomer () {
        Console.WriteLine("Add new customer");
    }
    
    public void DeleteLastCustomer () {
        Console.WriteLine("Last customer deleted!");
    }
}

public class AddCustomerCommand : ICommand {
    private CustomerService customerService;
    
    public AddCustomerCommand (CustomerService customerService) {
        this.customerService = customerService;
    }
    
    public void Execute () {
        customerService.AddCustomer();
    }
}

public class DeleteLastCustomerCommand : ICommand {
    private CustomerService customerService;
    
    public DeleteLastCustomerCommand (CustomerService customerService) {
        this.customerService = customerService;
    }
    
    public void Execute () {
        customerService.DeleteLastCustomer();
    }
}

In [14]:
var customerService = new CustomerService ();

var addNewCustomerButton = new Button (
    new AddCustomerCommand (customerService)
);
addNewCustomerButton.Click();

var deleteLastCustomerCommand = new Button (
    new DeleteLastCustomerCommand (customerService)
);
deleteLastCustomerCommand.Click();

Add new customer
Last customer deleted!


### Composite Command

In [18]:
public class CompositeCommand : ICommand {
    private List<ICommand> commands;
    
    public CompositeCommand () {
        commands = new List<ICommand>();
    }
    
    public void Add (ICommand command) {
        commands.Add(command);
    }
    
    public void Execute () {
        foreach (ICommand command in commands)
            command.Execute();
    }
}

public class SaveCommand : ICommand {
    public void Execute () {
        Console.WriteLine("Save");
    }
}

public class ResizeCommand : ICommand {
    public void Execute () {
        Console.WriteLine("Resize");
    }
}

public class MakeBlackWhiteCommand : ICommand {
    public void Execute () {
        Console.WriteLine("Make Black/White");
    }
}

In [19]:
var commands = new CompositeCommand ();
commands.Add(new ResizeCommand());
commands.Add(new MakeBlackWhiteCommand());
commands.Add(new SaveCommand());
commands.Execute();

Resize
Make Black/White
Save


### Undo Mechanism

In [30]:
public interface IUndoableCommand {
    void Unexecute();
}

public class HTMLDocument {
    public string Content { get; set; }
    
    public void MakeBold () {
        Content = "<b>" + Content + "</b>";
    }
}

public class History { 
    private Queue<IUndoableCommand> undoableCommands;

    public int Size { get {return undoableCommands.Count; } }
    
    public History () {
        undoableCommands = new Queue<IUndoableCommand>();
    }
    
    public void Push(IUndoableCommand command) {
        undoableCommands.Enqueue(command);
    }
    
    public IUndoableCommand Pop() {
        return undoableCommands.Dequeue();
    }
}

public class BoldCommand : ICommand, IUndoableCommand {
    private string previousContent;
    private HTMLDocument htmlDocument;
    private History history;
    
    public BoldCommand(HTMLDocument htmlDocument, History history) {
        this.htmlDocument = htmlDocument;
        this.history = history;            
    }
    
    public void Execute () {
        previousContent = htmlDocument.Content;
        htmlDocument.MakeBold();
        history.Push(this);        
    }
    
    public void Unexecute () {
        htmlDocument.Content = previousContent;
    }
}

In [31]:
var history = new History();
var document = new HTMLDocument();
document.Content = "Hi There!";
Console.WriteLine(document.Content);

var boldCommand = new BoldCommand(document, history);
boldCommand.Execute();
Console.WriteLine(document.Content);

boldCommand.Unexecute();
Console.WriteLine(document.Content);

Hi There!
<b>Hi There!</b>
Hi There!


In [32]:
public class UndoCommand : ICommand {
    private History history;
    
    public UndoCommand (History history) {
        this.history = history;
    }
    
    public void Execute () {
        if (history.Size > 0)
            history.Pop().Unexecute();
    }
}

In [33]:
var history = new History();
var document = new HTMLDocument();
document.Content = "Hi There!";
Console.WriteLine(document.Content);

var boldCommand = new BoldCommand(document, history);
boldCommand.Execute();
Console.WriteLine(document.Content);

var undoCommand = new UndoCommand(history);
undoCommand.Execute();
Console.WriteLine(document.Content);

Hi There!
<b>Hi There!</b>
Hi There!
